<a href="https://colab.research.google.com/github/dominicwhite/DeepFossil/blob/master/notebooks/01-Simple_FullyConnected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets, transforms


In [0]:
class FlattenImage():
    def __call__(self, t):
        return t.view((1,-1)).squeeze()

transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.Grayscale(1),
    transforms.ToTensor(),
    FlattenImage()
])
dataset = datasets.ImageFolder('/content/gdrive/My Drive/Colab Notebooks/data/CT/simulated_volumes/128', transform=transform)
dataset.classes

dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [0]:
images, classes = next(iter(dataloader))

In [29]:
im = images[1]
im.shape

torch.Size([1, 16384])

In [5]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [49]:
# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden = nn.Linear(128*128, 64*64)
        self.output = nn.Linear(64*64, 5)

    def forward(self, x):
        x = F.relu(self.hidden(x))
        x = F.log_softmax(self.output(x), dim=1)
        return x

model = Net()
print(model)
model.forward(images)

Net(
  (hidden): Linear(in_features=16384, out_features=4096, bias=True)
  (output): Linear(in_features=4096, out_features=5, bias=True)
)


RuntimeError: ignored

In [50]:
# create a complete CNN
model = Net()
print(model)

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

from torch import nn, optim

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 10

for e in range(epochs):
    print("Starting epoch", e)
#     running_loss = 0
    for images, labels in dataloader:
#         print(images.shape)
        images, labels = images.to('cuda', dtype=torch.float), labels.to('cuda', dtype=torch.long)
        optimizer.zero_grad()
        
        output = model(images)
#         print(output.shape)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
    else:
        ## TODO: Implement the validation pass and print out the validation accuracy
        with torch.no_grad():
            correct = 0
            total = torch.zeros(1)
            for images, labels in dataloader:
                images, labels = images.to('cuda', dtype=torch.float), labels.to('cuda', dtype=torch.long)
                ps = torch.exp(model(images))
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                correct += torch.sum(equals)
                total += len(equals)
#                 accuracy = accuracy + torch.mean(equals.type(torch.FloatTensor))
            accuracy = correct / total
        print(f'Accuracy: {accuracy.item()*100}%')


Net(
  (hidden): Linear(in_features=16384, out_features=4096, bias=True)
  (output): Linear(in_features=4096, out_features=5, bias=True)
)
Starting epoch 0
Accuracy: 72.5781261920929%
Starting epoch 1
Accuracy: 78.67187261581421%
Starting epoch 2
Accuracy: 73.67187738418579%
Starting epoch 3
Accuracy: 81.9531261920929%
Starting epoch 4
Accuracy: 85.2343738079071%
Starting epoch 5
Accuracy: 88.28125%
Starting epoch 6
Accuracy: 82.3437511920929%
Starting epoch 7
Accuracy: 89.29687738418579%
Starting epoch 8
Accuracy: 80.7812511920929%
Starting epoch 9
Accuracy: 90.78124761581421%


In [0]:
# images, labels = next(iter(dataloader))
# images = images.to('cuda')
# model(images)

In [0]:
# images[0][0,:,:]